# Learning database queries via intelligent semiotic machines

Publication information: Martins, D. M. L., Vossen, G., & de Lima Neto, F. B. (2017, November). Learning database queries via intelligent semiotic machines. In 2017 IEEE Latin American Conference on Computational Intelligence (LA-CCI) (pp. 1-6). IEEE.

URL: https://ieeexplore.ieee.org/document/8285698

Publication's BibTeX:

### Configuring notebook

In [2]:
cd ..

C:\Users\d_mart04\Documents\CodeRepositoryGit\enablingnontechsdb\experiments


In [3]:
cd ..

C:\Users\d_mart04\Documents\CodeRepositoryGit\enablingnontechsdb


In [4]:
import sys, os
sys.path.append('..\\sada')
sys.path.append('..\\decision')
sys.path.append('..\\qbe')

In [5]:
from datamanagement.dataaccessobject import DataAccessObject, Dataset
from decision.somselector import SomSelector
from qbe.treeqbe import DecisionTreeQBE
from sada.decisionsada import DecisionSADA
from qbe import util
import pandas as pd, pandasql as pdsql

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support

### Loading car dataset

In [7]:
DAO = DataAccessObject()
dataset = DAO.get_car_dataset()

C:\Users\d_mart04\Documents\CodeRepositoryGit\enablingnontechsdb\datamanagement\dataaccessobject.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  self.data_matrix = preprocessed_data.as_matrix()


### Configuring SADA

In [8]:
sada = DecisionSADA(dataset, selector=SomSelector(som_size=(10,10), num_iterations=1000))

### Car dataset

In [9]:
dataset.original_data.head()

,make,manufacturer,type,price,mpg,num_of_cylinders,horsepower,fuel_tank_capacity,RPM,Wheelbase,...,Weight,automatic_gearbox,passenger_capacity,length,width,luggage_capacity,AirBags,DriveTrain,Origin,imagepath
1,Acura Integra,Acura,Small,7950.0,28.0,4,140,13.2,6300,102,...,2705,0,5,177,68,11.0,None,Front,1,01acura_integra.jpg
2,Acura Legend,Acura,Midsize,16950.0,21.5,6,200,18.0,5500,115,...,3560,0,5,195,71,15.0,DriverAndPassenger,Front,1,02Acura_Legend.jpg
3,Audi 90,Audi,Compact,14550.0,23.0,6,172,16.9,5500,102,...,3375,0,5,180,67,14.0,DriverOnly,Front,1,03audi_100.jpg
4,Audi 100,Audi,Midsize,18850.0,22.5,6,172,21.1,5500,106,...,3405,0,6,193,70,17.0,DriverAndPassenger,Front,1,04audi_90.jpg
5,BMW 535i,BMW,Midsize,15000.0,26.0,4,208,21.1,5700,109,...,3640,0,4,186,69,13.0,DriverOnly,Rear,1,05bmw_535i.jpg


In [10]:
concept = dataset.original_data.query('type=="Sporty" and Origin ==0')
print(concept.index)
y_test = [int(y in concept.index) for y in range(1, dataset.original_data.shape[0]+1)]
print(y_test)

Int64Index([14, 19, 28, 34, 35, 60, 72, 75], dtype='int64')
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


### Select an example index to start the learning process

In [12]:
example_index = 33
tuple_input = dataset.data_matrix[example_index].tolist()
dataset.original_data.iloc[example_index]

make                        Ford Mustang
manufacturer                        Ford
type                              Sporty
price                               7950
mpg                                 25.5
num_of_cylinders                       4
horsepower                           105
fuel_tank_capacity                  15.4
RPM                                 4600
Wheelbase                            101
Rear.seat.room                        24
Weight                              2850
automatic_gearbox                      0
passenger_capacity                     4
length                               180
width                                 68
luggage_capacity                      12
AirBags                       DriverOnly
DriveTrain                          Rear
Origin                                 0
imagepath             34ford_mustang.jpg
Name: 34, dtype: object

### Selected examples

In [13]:
selected_indexes = sada.select(query=tuple_input)
print(selected_indexes)

[33, 27, 56, 59, 13]


In [15]:
predicted = [int(i in selected_indexes) for i in range(dataset.original_data.shape[0])]

In [16]:
precision_recall_fscore_support(y_test, predicted, average='binary')

(0.8, 0.5, 0.6153846153846154, None)

### Query learning phase

In [17]:
# Copying dataset to a dataframe
dataframe = dataset.preprocessed_data.copy(deep=True)

# Configure PandaSQL to query Pandas dataframe
pysql = lambda q: pdsql.sqldf(q, globals())

In [18]:
query_learner = DecisionTreeQBE(dataframe, selected_indexes)
best_predicate = query_learner.search_best_predicate()

In [19]:
learned_view = util.get_view_from_predicate(best_predicate, dataframe, dataframe.iloc[selected_indexes], pysql)

Query:  SELECT * FROM dataframe WHERE rear_seat_room > 0.14705882407724857 AND num_of_cylinders > 0.1875 AND Sporty > 0.5 AND RPM <= 0.37037037312984467 AND width <= 0.8055555522441864 OR rear_seat_room > 0.14705882407724857 AND num_of_cylinders <= 0.1875 OR rear_seat_room <= 0.14705882407724857
---------------------------------
Relevant & Actual:  5
Recall:  1.0
Specificity:  1.0
Precision:  1.0
F1-Score:  1.0
False negative rate:  0.0
False positive rate:  0.0
---------------------------------


### Show retrieved data

In [20]:
original_indexes = util.get_original_indexes_from_view(dataframe, learned_view)

In [23]:
dataset.original_data.iloc[original_indexes].head()

,make,manufacturer,type,price,mpg,num_of_cylinders,horsepower,fuel_tank_capacity,RPM,Wheelbase,...,Weight,automatic_gearbox,passenger_capacity,length,width,luggage_capacity,AirBags,DriveTrain,Origin,imagepath
14,Chevrolet Camaro,Chevrolet,Sporty,7550.0,23.5,6,160,15.5,4600,101,...,3240,0,4,193,74,13.000000,DriverAndPassenger,Rear,0,14chevrolet_camaro.jpg
28,Dodge Stealth,Dodge,Sporty,12900.0,21.0,6,300,19.8,6000,97,...,3805,0,4,180,72,11.000000,DriverOnly,4WD,0,28dodge_stealth.jpg
34,Ford Mustang,Ford,Sporty,7950.0,25.5,4,105,15.4,4600,101,...,2850,0,4,180,68,12.000000,DriverOnly,Rear,0,34ford_mustang.jpg
57,Mazda RX-7,Mazda,Sporty,16250.0,21.0,0,255,20.0,6500,96,...,2895,0,2,169,69,13.506216,DriverOnly,Rear,1,57mazda_rx7.jpg
60,Mercury Capri,Mercury,Sporty,7050.0,24.5,4,100,11.1,5750,95,...,2450,0,4,166,65,6.000000,DriverOnly,Front,0,60mercury_capri.jpg


In [24]:
queries = ["type == 'Sporty' and Origin == 0", 
    "type != 'Sporty' and Origin == 1",
    "automatic_gearbox == 1 and horsepower >= 150",
    "luggage_capacity >= 18 and passenger_capacity > 5",
    "price <= 7000 and mpg >= 26 and automatic_gearbox == 0",
    "manufacturer == 'Ford' or manufacturer == 'Chevrolet'"]

In [38]:
results = []
for q in queries:
    concept = dataset.original_data.query(q)
    y_test = [int(y in concept.index) for y in range(1, dataset.original_data.shape[0]+1)]
    sada = DecisionSADA(dataset, selector=SomSelector(som_size=(10,10), num_iterations=dataset.original_data.shape[0]*10))
    for i in range(10):
        example_index = random.choice(concept.index)
        tuple_input = dataset.data_matrix[example_index-1].tolist()
        selected_indexes = sada.select(query=tuple_input,  num_of_selected_candidates=len(concept))
        predicted = [int(i in selected_indexes) for i in range(dataset.original_data.shape[0])]
        res = [q, len(concept.index)] + list(precision_recall_fscore_support(y_test, predicted, average='binary'))[:3]
        results.append(res)

In [39]:
df = pd.DataFrame(results, columns=['query', 'concept size', 'precision', 'recall', 'f1score'])

In [40]:
df.groupby(by=['query', 'concept size']).mean().sort_values(by='f1score', ascending=False)

,,precision,recall,f1score
query,concept size,,,
price <= 7000 and mpg >= 26 and automatic_gearbox == 0,25,0.636000,0.636000,0.636000
type == 'Sporty' and Origin == 0,8,0.550000,0.550000,0.550000
automatic_gearbox == 1 and horsepower >= 150,23,0.439130,0.439130,0.439130
type != 'Sporty' and Origin == 1,39,0.407692,0.407692,0.407692
luggage_capacity >= 18 and passenger_capacity > 5,9,0.366667,0.366667,0.366667
manufacturer == 'Ford' or manufacturer == 'Chevrolet',16,0.268750,0.268750,0.268750
